# Create Slice for Fabric Rack
This is the slice creation for the ```Webserver``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "MyWebServerSlice"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(slice_name)
    
    # deleted spaces in the calls
    
    # Add node
    server = slice.add_node(name='Server', site=site)
    server.set_capacities(cores=cores, ram=ram, disk=disk)
    server.set_image(image)
    ifaces1 = server.add_component(model='NIC_Basic', name='server-nic1').get_interfaces()[0] 
    ifaces2 = server.add_component(model='NIC_Basic', name='server-nic2').get_interfaces()[0] 
    
    # Add node
    node1 = slice.add_node(name='Client1', site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    ifacen1 = node1.add_component(model='NIC_Basic', name='node1-nic1').get_interfaces()[0] 
    
    # Add node
    node2 = slice.add_node(name='Client2', site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    ifacen2 = node2.add_component(model='NIC_Basic', name='node2-nic1').get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name='bridge1', interfaces=[ifaces1, ifacen1])
    net2 = slice.add_l2network(name='bridge2', interfaces=[ifaces2, ifacen2])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will asign the network spaces some predefined values that we will use for the Assgnment

In [ ]:
# Setup Network

try:
    server_name  = 'Server'
    node_1_name = 'Client1'
    node_2_name = 'Client2'

    network_service_name1 = "bridge1"
    network_service_name2 = "bridge2"

    server_ip1 = '10.20.30.40'
    server_ip2 = '10.20.30.41'
    node1_ip = '10.20.30.42'
    node2_ip = '10.20.30.43'
    
    server = slice.get_node(name=server_name)        
    ifaces1 = server.get_interface(network_name=network_service_name1)  
    ifaces1.set_ip(ip=server_ip1, cidr="24")
    
    ifaces2 = server.get_interface(network_name=network_service_name2)  
    ifaces2.set_ip(ip=server_ip2, cidr="24")
    
    Device_P1 = ifaces1.get_device_name()
    Device_P2 = ifaces2.get_device_name()
    
    #Node1 conection
    stdout, stderr = server.execute(f'ip addr show {ifaces1.get_os_interface()}')
    server.execute(f"sudo ip link set dev {Device_P1} up")
    print (stdout)
    #node2 conection
    stdout, stderr = server.execute(f'ip addr show {ifaces2.get_os_interface()}')
    server.execute(f"sudo ip link set dev { Device_P2 } up")
    print (stdout)
    
    node1 = slice.get_node(name=node_1_name)        
    ifacen1 = node1.get_interface(network_name=network_service_name1)  
    ifacen1.set_ip(ip=node1_ip, cidr="24")
    
    stdout, stderr = node1.execute(f'ip addr show {ifacen1.get_os_interface()}')
    node1.execute(f"sudo ip link set dev { ifacen1.get_device_name()} up")
    print (stdout)
    
    node2 = slice.get_node(name=node_2_name)        
    ifacen2 = node2.get_interface(network_name=network_service_name2)
    ifacen2.set_ip(ip=node2_ip, cidr="24")
    stdout, stderr = node2.execute(f'ip addr show {ifacen2.get_os_interface()}')
    node2.execute(f"sudo ip link set dev { ifacen2.get_device_name()} up")
    print (stdout)
    
    # 1 packets transmitted, 0 received, +1 errors, 100% packet loss, time 0ms
    # swamp the words "enp7s0" and "enp8s0" this just means that the ips were created on oposite devices
    
    server.execute(f'sudo route del -net 10.20.30.0 gw 0.0.0.0 netmask 255.255.255.0 dev {Device_P2}')
    server.execute(f'sudo route del -net 10.20.30.0 gw 0.0.0.0 netmask 255.255.255.0 dev {Device_P1}')
    server.execute(f'sudo ip route add {node1_ip} dev {Device_P1}')
    server.execute(f'sudo ip route add {node2_ip} dev {Device_P2}')
    print ("Testing Connection...")
    server.execute(f'ping 10.20.30.43 -c 1')
    server.execute(f'ping 10.20.30.42 -c 1')
    
except Exception as e:
    print(f"Error: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will Add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
try:
    print("Installing tools testing tools")
    stdout, stderr = server.execute(f'sudo apt-get update', quiet=False)
    stdout, stderr = server.execute(f'sudo apt-get install net-tools', quiet=False)
    
    server = slice.get_node(name=server_name)        
    server.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/Webserver/webexample.tar.gz")
    server.execute("tar xvfz webexample.tar.gz")

    server.execute("sudo ./webexample/websrv_init.sh")
    server.execute("sudo cp -avr webexample/website ../../var/www")
    
    #node 1
    node1.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/Webserver/webexample.tar.gz")
    node1.execute("tar xvfz webexample.tar.gz")
    node1.execute("sudo ./webexample/webcl_init.sh")
    
    #node2    
    node2.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/Webserver/webexample.tar.gz")
    node2.execute("tar xvfz webexample.tar.gz")
    node2.execute("sudo ./webexample/webcl_init.sh")

except Exception as e:
    print(f"Error: {e}")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Webserver, you can ether open it on the explorer or click [Here](./webserver.ipynb) to open the next notebook.